In [1]:
# source activate python2
# conda install -c menpo opencv3

In [1]:
from cv2 import __version__
__version__

'3.1.0'

In [2]:
import cv2
import joblib
import os
import csv
import numpy
import sklearn
from sklearn import svm
import time
from random import randint


In [3]:
os.getcwd()
os.chdir('/Users/yueyingteng/Documents/2016.9/Big Data /kaggle')

In [4]:
folderPath='./fish/ALB'
ALB_index = set(range(2544))
image_name = os.listdir(folderPath)
image_names = image_name[0:len(image_name)-1]
image_names=[image_names[i] for i in ALB_index]
image_paths = [os.path.join(folderPath, f) for f in image_names]

In [5]:
# feature_det = cv2.FeatureDetector_create("SIFT")
feature_det = cv2.xfeatures2d.SIFT_create()

def preProcessImages(image_paths):
    descriptors= []
    for image_path in image_paths:
        im = cv2.imread(image_path)
        kpts = feature_det.detect(im)
        # kpts, des = descr_ext.compute(im, kpts)
        kpts, des = feature_det.compute(im, kpts)
        descriptors.append(des)
    return descriptors

In [6]:
start = time.time()
descriptors = preProcessImages(image_paths)
end = time.time()
print "minutes spent in Descriptors"
print (end - start)/60

minutes spent in Descriptors
1.09332924684


In [7]:
descriptors_none=[]
for i, j in enumerate(descriptors):
    if j == None:
        descriptors_none.append(i)

descriptors=[i for i in descriptors if i!= None]
# image_classes_train=[image_classes_train[i] for i in range(len(image_classes_train)) if i not in descriptors_none]
image_paths=[image_paths[i] for i in range(len(image_paths)) if i not in descriptors_none]


/Users/yueyingteng/anaconda3/envs/python2/lib/python2.7/site-packages/ipykernel/__main__.py:3: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  app.launch_new_instance()
/Users/yueyingteng/anaconda3/envs/python2/lib/python2.7/site-packages/ipykernel/__main__.py:6: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.


In [8]:
# def train(descriptors,image_classes_train,image_paths):  
flann_params = dict(algorithm = 1, trees = 5)     
matcher = cv2.FlannBasedMatcher(flann_params, {})

# extract descriptor of the new images baesed on the constructed vocabulary
# bow_extract  =cv2.BOWImgDescriptorExtractor(descr_ext,matcher)
bow_extract  =cv2.BOWImgDescriptorExtractor(feature_det,matcher)

def getImagedata(feature_det,bow_extract,path):
    im = cv2.imread(path)
    featureset = bow_extract.compute(im, feature_det.detect(im))
    return featureset
# returned featureset contains normzlized histogram

In [9]:
# clustering k=500
bow_train = cv2.BOWKMeansTrainer(5)

# create the vocabulary
for des in descriptors:
    bow_train.add(des)

In [14]:
start = time.time()
voc = bow_train.cluster()
bow_extract.setVocabulary(voc)

end = time.time()
print "minutes spent in creating Vocabulary"
print (end - start)/60

In [ ]:
# get features from the training data based on the vocabulary & approximate nearest neighbour
# features are used as the training data 
traindata = []  

start = time.time()
for imagepath in image_paths:
    featureset = getImagedata(feature_det,bow_extract,imagepath)
    traindata.extend(featureset)

end = time.time()
print ("minutes spent in Extracting vocabulary")
print ((end - start)/60)

In [ ]:
# train linear svm
clf=sklearn.svm.classes.LinearSVC()

# clf = LinearSVC()
clf.fit(traindata, numpy.array(image_classes_train))
joblib.dump((voc,clf), "imagereco.pkl", compress=3)

# model=train(descriptors,image_classes_train,image_paths)
voc, clf = joblib.load("imagereco.pkl")


X=numpy.matrix(traindata)
numpy.savetxt('X2.csv', X,delimiter=',')

my_file = open("label_2_2.txt", "w")
for item in image_classes_train:
    my_file.write("%s\n" % item)

my_file.close()

my_file = open("animal_names_2.txt", "w")
for item in training_names:
    my_file.write("%s\n" % item)

my_file.close()